# Oracle AI 向量搜索与文档处理
Oracle AI 向量搜索是专为人工智能(AI)工作负载设计的，它允许您基于语义而不是关键词来查询数据。
Oracle AI 向量搜索最大的优势之一是可以在单个系统中将非结构化数据的语义搜索与业务数据的关系搜索相结合。
这不仅功能强大，而且显著提高了效率，因为您无需添加专门的向量数据库，从而避免了多系统之间的数据碎片化问题。

此外，您的向量可以受益于 Oracle 数据库的所有强大功能，例如：

 * [分区支持](https://www.oracle.com/database/technologies/partitioning.html)
 * [Real Application Clusters 可扩展性](https://www.oracle.com/database/real-application-clusters/)
 * [Exadata 智能扫描](https://www.oracle.com/database/technologies/exadata/software/smartscan/)
 * [跨地理分布数据库的分片处理](https://www.oracle.com/database/distributed-database/)
 * [事务处理](https://docs.oracle.com/en/database/oracle/oracle-database/23/cncpt/transactions.html)
 * [并行 SQL](https://docs.oracle.com/en/database/oracle/oracle-database/21/vldbg/parallel-exec-intro.html#GUID-D28717E4-0F77-44F5-BB4E-234C31D4E4BA)
 * [灾难恢复](https://www.oracle.com/database/data-guard/)
 * [安全性](https://www.oracle.com/security/database-security/)
 * [Oracle 机器学习](https://www.oracle.com/artificial-intelligence/database-machine-learning/)
 * [Oracle 图数据库](https://www.oracle.com/database/integrated-graph-database/)
 * [Oracle 空间和图形](https://www.oracle.com/database/spatial/)
 * [Oracle 区块链](https://docs.oracle.com/en/database/oracle/oracle-database/23/arpls/dbms_blockchain_table.html#GUID-B469E277-978E-4378-A8C1-26D3FF96C9A6)
 * [JSON](https://docs.oracle.com/en/database/oracle/oracle-database/23/adjsn/json-in-oracle-database.html)

本指南演示了如何使用 Oracle AI 向量搜索与 Langchain 来实现端到端的 RAG 管道。本指南包含以下示例：

 * 使用 OracleDocLoader 从各种来源加载文档
 * 使用 OracleSummary 在数据库内部/外部对文档进行摘要
 * 使用 OracleEmbeddings 在数据库内部/外部生成文档的嵌入向量
 * 使用 OracleTextSplitter 中的高级 Oracle 功能根据不同需求对文档进行分块
 * 在向量存储中存储和索引文档，并使用 OracleVS 进行查询

如果您刚开始使用 Oracle 数据库，建议探索 [免费的 Oracle 23 AI](https://www.oracle.com/database/free/#resources)，它为设置数据库环境提供了很好的入门指南。在使用数据库时，建议避免默认使用系统用户；相反，您可以创建自己的用户以增强安全性和自定义性。有关用户创建的详细步骤，请参阅我们的[端到端指南](https://github.com/langchain-ai/langchain/blob/master/cookbook/oracleai_demo.ipynb)，其中还展示了如何在 Oracle 中设置用户。此外，了解用户权限对于有效管理数据库安全至关重要。您可以在官方 [Oracle 指南](https://docs.oracle.com/en/database/oracle/oracle-database/19/admqs/administering-user-accounts-and-security.html#GUID-36B21D72-1BBB-46C9-A0C9-F0D2A8591B8D)中了解更多关于管理用户账户和安全性的信息。

### 前提条件

请安装 Oracle Python 客户端驱动程序以使用 Langchain 和 Oracle AI 向量搜索。

In [ ]:
# pip install oracledb

### 创建演示用户
首先，创建一个具有所有必需权限的演示用户。

In [37]:
import sys

import oracledb

# Update with your username, password, hostname, and service_name
username = ""
password = ""
dsn = ""

try:
    conn = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Connection successful!")

    cursor = conn.cursor()
    try:
        cursor.execute(
            """
            begin
                -- Drop user
                begin
                    execute immediate 'drop user testuser cascade';
                exception
                    when others then
                        dbms_output.put_line('Error dropping user: ' || SQLERRM);
                end;
                
                -- Create user and grant privileges
                execute immediate 'create user testuser identified by testuser';
                execute immediate 'grant connect, unlimited tablespace, create credential, create procedure, create any index to testuser';
                execute immediate 'create or replace directory DEMO_PY_DIR as ''/scratch/hroy/view_storage/hroy_devstorage/demo/orachain''';
                execute immediate 'grant read, write on directory DEMO_PY_DIR to public';
                execute immediate 'grant create mining model to testuser';
                
                -- Network access
                begin
                    DBMS_NETWORK_ACL_ADMIN.APPEND_HOST_ACE(
                        host => '*',
                        ace => xs$ace_type(privilege_list => xs$name_list('connect'),
                                           principal_name => 'testuser',
                                           principal_type => xs_acl.ptype_db)
                    );
                end;
            end;
            """
        )
        print("User setup done!")
    except Exception as e:
        print(f"User setup failed with error: {e}")
    finally:
        cursor.close()
    conn.close()
except Exception as e:
    print(f"Connection failed with error: {e}")
    sys.exit(1)

Connection successful!
User setup done!


## 使用 Oracle AI 处理文档
考虑以下场景：用户拥有存储在 Oracle 数据库或文件系统中的文档，并打算将这些数据与由 Langchain 支持的 Oracle AI 向量搜索一起使用。

要准备文档进行分析，需要一个全面的预处理工作流程。首先，必须检索文档，根据需要生成摘要，并按需要进行分块。后续步骤包括为这些块生成嵌入向量，并将它们集成到 Oracle AI 向量存储中。然后，用户可以对这些数据进行语义搜索。

Oracle AI 向量搜索 Langchain 库包含一套文档处理工具，用于文档加载、分块、摘要生成和嵌入向量创建。

在接下来的部分中，我们将详细说明如何使用 Oracle AI Langchain API 有效实施这些过程。

### 连接到演示用户
以下示例代码将展示如何连接到 Oracle 数据库。默认情况下，python-oracledb 以'精简'模式运行，直接连接到 Oracle 数据库。这种模式不需要 Oracle 客户端库。但是，当 python-oracledb 使用客户端库时，可以使用一些额外的功能。当使用 Oracle 客户端库时，python-oracledb 被称为'完整'模式。两种模式都具有全面的功能，支持 Python 数据库 API v2.0 规范。请参阅以下[指南](https://python-oracledb.readthedocs.io/en/latest/user_guide/appendix_a.html#featuresummary)，其中介绍了每种模式支持的功能。如果您无法使用精简模式，可能需要切换到完整模式。

In [45]:
import sys

import oracledb

# please update with your username, password, hostname and service_name
username = ""
password = ""
dsn = ""

try:
    conn = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Connection successful!")
except Exception as e:
    print("Connection failed!")
    sys.exit(1)

Connection successful!


### 填充演示表
创建一个演示表并插入一些示例文档。

In [46]:
try:
    cursor = conn.cursor()

    drop_table_sql = """drop table demo_tab"""
    cursor.execute(drop_table_sql)

    create_table_sql = """create table demo_tab (id number, data clob)"""
    cursor.execute(create_table_sql)

    insert_row_sql = """insert into demo_tab values (:1, :2)"""
    rows_to_insert = [
        (
            1,
            "If the answer to any preceding questions is yes, then the database stops the search and allocates space from the specified tablespace; otherwise, space is allocated from the database default shared temporary tablespace.",
        ),
        (
            2,
            "A tablespace can be online (accessible) or offline (not accessible) whenever the database is open.\nA tablespace is usually online so that its data is available to users. The SYSTEM tablespace and temporary tablespaces cannot be taken offline.",
        ),
        (
            3,
            "The database stores LOBs differently from other data types. Creating a LOB column implicitly creates a LOB segment and a LOB index. The tablespace containing the LOB segment and LOB index, which are always stored together, may be different from the tablespace containing the table.\nSometimes the database can store small amounts of LOB data in the table itself rather than in a separate LOB segment.",
        ),
    ]
    cursor.executemany(insert_row_sql, rows_to_insert)

    conn.commit()

    print("Table created and populated.")
    cursor.close()
except Exception as e:
    print("Table creation failed.")
    cursor.close()
    conn.close()
    sys.exit(1)

Table created and populated.


随着演示用户和示例表的创建，剩下的配置涉及设置嵌入和摘要功能。用户可以选择多种提供者选项，包括本地数据库解决方案和第三方服务，如 Ocigenai、Hugging Face 和 OpenAI。如果用户选择第三方提供者，则需要建立包含必要身份验证详细信息的凭据。相反，如果选择数据库作为嵌入提供者，则需要将 ONNX 模型上传到 Oracle 数据库。使用数据库选项时，摘要功能不需要额外设置。

### 加载 ONNX 模型

Oracle 支持多种嵌入提供者，使用户能够在专有数据库解决方案和第三方服务（如 OCIGENAI 和 HuggingFace）之间进行选择。这种选择决定了生成和管理嵌入向量的方法。

***重要提示*** ：如果用户选择数据库选项，必须将 ONNX 模型上传到 Oracle 数据库。相反，如果选择第三方提供者来生成嵌入向量，则不需要将 ONNX 模型上传到 Oracle 数据库。

直接在 Oracle 中使用 ONNX 模型的一个显著优势是，通过消除向外部方传输数据的需求，它提供了增强的安全性和性能。此外，这种方法避免了通常与网络或 REST API 调用相关的延迟。

以下是将 ONNX 模型上传到 Oracle 数据库的示例代码：

In [47]:
from langchain_community.embeddings.oracleai import OracleEmbeddings

# please update with your related information
# make sure that you have onnx file in the system
onnx_dir = "DEMO_PY_DIR"
onnx_file = "tinybert.onnx"
model_name = "demo_model"

try:
    OracleEmbeddings.load_onnx_model(conn, onnx_dir, onnx_file, model_name)
    print("ONNX model loaded.")
except Exception as e:
    print("ONNX model loading failed!")
    sys.exit(1)

ONNX model loaded.


### 创建凭据

当选择第三方提供者生成嵌入向量时，用户需要建立凭据以安全访问提供者的端点。

***重要提示：*** 当选择'数据库'提供者生成嵌入向量时，不需要凭据。但是，如果用户决定使用第三方提供者，则必须为所选提供者创建特定的凭据。

以下是一个示例：

In [ ]:
try:
    cursor = conn.cursor()
    cursor.execute(
        """
       declare
           jo json_object_t;
       begin
           -- HuggingFace
           dbms_vector_chain.drop_credential(credential_name  => 'HF_CRED');
           jo := json_object_t();
           jo.put('access_token', '<access_token>');
           dbms_vector_chain.create_credential(
               credential_name   =>  'HF_CRED',
               params            => json(jo.to_string));

           -- OCIGENAI
           dbms_vector_chain.drop_credential(credential_name  => 'OCI_CRED');
           jo := json_object_t();
           jo.put('user_ocid','<user_ocid>');
           jo.put('tenancy_ocid','<tenancy_ocid>');
           jo.put('compartment_ocid','<compartment_ocid>');
           jo.put('private_key','<private_key>');
           jo.put('fingerprint','<fingerprint>');
           dbms_vector_chain.create_credential(
               credential_name   => 'OCI_CRED',
               params            => json(jo.to_string));
       end;
       """
    )
    cursor.close()
    print("Credentials created.")
except Exception as ex:
    cursor.close()
    raise

### 加载文档
用户可以通过适当配置加载器参数，灵活地从 Oracle 数据库、文件系统或两者中加载文档。有关这些参数的详细信息，请参阅 [Oracle AI 向量搜索指南](https://docs.oracle.com/en/database/oracle/oracle-database/23/arpls/dbms_vector_chain1.html#GUID-73397E89-92FB-48ED-94BB-1AD960C4EA1F)。

使用 OracleDocLoader 的一个显著优势是它能够处理超过 150 种不同的文件格式，消除了为不同文档类型使用多个加载器的需求。有关支持的格式的完整列表，请参阅 [Oracle Text 支持的文档格式](https://docs.oracle.com/en/database/oracle/oracle-database/23/ccref/oracle-text-supported-document-formats.html)。

以下是展示如何使用 OracleDocLoader 的示例代码

In [48]:
from langchain_community.document_loaders.oracleai import OracleDocLoader
from langchain_core.documents import Document

# loading from Oracle Database table
# make sure you have the table with this specification
loader_params = {}
loader_params = {
    "owner": "testuser",
    "tablename": "demo_tab",
    "colname": "data",
}

""" load the docs """
loader = OracleDocLoader(conn=conn, params=loader_params)
docs = loader.load()

""" verify """
print(f"Number of docs loaded: {len(docs)}")
# print(f"Document-0: {docs[0].page_content}") # content

Number of docs loaded: 3


### 生成摘要
现在用户已经加载了文档，他们可能想要为每个文档生成摘要。Oracle AI 向量搜索 Langchain 库提供了一套用于文档摘要的 API。它支持多个摘要提供者，如数据库、OCIGENAI、HuggingFace 等，允许用户选择最适合其需求的提供者。要使用这些功能，用户必须按照指定配置摘要参数。有关这些参数的详细信息，请参阅 [Oracle AI 向量搜索指南书](https://docs.oracle.com/en/database/oracle/oracle-database/23/arpls/dbms_vector_chain1.html#GUID-EC9DDB58-6A15-4B36-BA66-ECBA20D2CE57)。

***注意：*** 如果用户想要使用除 Oracle 内置默认提供者'数据库'之外的第三方摘要生成提供者，可能需要设置代理。如果您没有代理，请在实例化 OracleSummary 时删除代理参数。

以下示例代码将展示如何生成摘要：

In [22]:
# proxy to be used when we instantiate summary and embedder object
proxy = ""

The following sample code will show how to generate summary:

In [49]:
from langchain_community.utilities.oracleai import OracleSummary
from langchain_core.documents import Document

# using 'database' provider
summary_params = {
    "provider": "database",
    "glevel": "S",
    "numParagraphs": 1,
    "language": "english",
}

# get the summary instance
# Remove proxy if not required
summ = OracleSummary(conn=conn, params=summary_params, proxy=proxy)

list_summary = []
for doc in docs:
    summary = summ.get_summary(doc.page_content)
    list_summary.append(summary)

""" verify """
print(f"Number of Summaries: {len(list_summary)}")
# print(f"Summary-0: {list_summary[0]}") #content

Number of Summaries: 3


### 拆分文档
文档的大小可能有所不同，从小到非常大。用户通常倾向于将文档分成较小的部分以便于生成嵌入向量。这个拆分过程有多种自定义选项可用。有关这些参数的详细信息，请参阅 [Oracle AI 向量搜索指南](https://docs.oracle.com/en/database/oracle/oracle-database/23/arpls/dbms_vector_chain1.html#GUID-4E145629-7098-4C7C-804F-FC85D1F24240)。

以下是一个示例代码，说明如何实现这一点：

In [ ]:
from langchain_community.document_loaders.oracleai import OracleTextSplitter
from langchain_core.documents import Document

# split by default parameters
splitter_params = {"normalize": "all"}

""" get the splitter instance """
splitter = OracleTextSplitter(conn=conn, params=splitter_params)

list_chunks = []
for doc in docs:
    chunks = splitter.split_text(doc.page_content)
    list_chunks.extend(chunks)

""" verify """
print(f"Number of Chunks: {len(list_chunks)}")
# print(f"Chunk-0: {list_chunks[0]") # content

Number of Chunks: 3


### 生成嵌入向量
现在文档已根据需求进行了分块，用户可能想要为这些块生成嵌入向量。Oracle AI 向量搜索提供了多种生成嵌入向量的方法，可以使用本地托管的 ONNX 模型或第三方 API。有关配置这些替代方案的详细说明，请参阅 [Oracle AI 向量搜索指南](https://docs.oracle.com/en/database/oracle/oracle-database/23/arpls/dbms_vector_chain1.html#GUID-C6439E94-4E86-4ECD-954E-4B73D53579DE)。

***注意：*** 用户可能需要配置代理才能使用第三方嵌入向量生成提供者，但使用'数据库'提供者（使用 ONNX 模型）时除外。

In [12]:
# proxy to be used when we instantiate summary and embedder object
proxy = ""

以下示例代码将展示如何生成嵌入向量：

In [51]:
from langchain_community.embeddings.oracleai import OracleEmbeddings
from langchain_core.documents import Document

# using ONNX model loaded to Oracle Database
embedder_params = {"provider": "database", "model": "demo_model"}

# get the embedding instance
# Remove proxy if not required
embedder = OracleEmbeddings(conn=conn, params=embedder_params, proxy=proxy)

embeddings = []
for doc in docs:
    chunks = splitter.split_text(doc.page_content)
    for chunk in chunks:
        embed = embedder.embed_query(chunk)
        embeddings.append(embed)

""" verify """
print(f"Number of embeddings: {len(embeddings)}")
# print(f"Embedding-0: {embeddings[0]}") # content

Number of embeddings: 3


## 创建 Oracle AI 向量存储
现在您已经了解了如何单独使用 Oracle AI Langchain 库 API 来处理文档，让我们展示如何与 Oracle AI 向量存储集成以实现语义搜索。

首先，让我们导入所有依赖项。

In [52]:
import sys

import oracledb
from langchain_community.document_loaders.oracleai import (
    OracleDocLoader,
    OracleTextSplitter,
)
from langchain_community.embeddings.oracleai import OracleEmbeddings
from langchain_community.utilities.oracleai import OracleSummary
from langchain_community.vectorstores import oraclevs
from langchain_community.vectorstores.oraclevs import OracleVS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_core.documents import Document

Next, let's combine all document processing stages together. Here is the sample code below:

In [53]:
"""
In this sample example, we will use 'database' provider for both summary and embeddings.
So, we don't need to do the followings:
    - set proxy for 3rd party providers
    - create credential for 3rd party providers

If you choose to use 3rd party provider, 
please follow the necessary steps for proxy and credential.
"""

# oracle connection
# please update with your username, password, hostname, and service_name
username = ""
password = ""
dsn = ""

try:
    conn = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Connection successful!")
except Exception as e:
    print("Connection failed!")
    sys.exit(1)


# load onnx model
# please update with your related information
onnx_dir = "DEMO_PY_DIR"
onnx_file = "tinybert.onnx"
model_name = "demo_model"
try:
    OracleEmbeddings.load_onnx_model(conn, onnx_dir, onnx_file, model_name)
    print("ONNX model loaded.")
except Exception as e:
    print("ONNX model loading failed!")
    sys.exit(1)


# params
# please update necessary fields with related information
loader_params = {
    "owner": "testuser",
    "tablename": "demo_tab",
    "colname": "data",
}
summary_params = {
    "provider": "database",
    "glevel": "S",
    "numParagraphs": 1,
    "language": "english",
}
splitter_params = {"normalize": "all"}
embedder_params = {"provider": "database", "model": "demo_model"}

# instantiate loader, summary, splitter, and embedder
loader = OracleDocLoader(conn=conn, params=loader_params)
summary = OracleSummary(conn=conn, params=summary_params)
splitter = OracleTextSplitter(conn=conn, params=splitter_params)
embedder = OracleEmbeddings(conn=conn, params=embedder_params)

# process the documents
chunks_with_mdata = []
for id, doc in enumerate(docs, start=1):
    summ = summary.get_summary(doc.page_content)
    chunks = splitter.split_text(doc.page_content)
    for ic, chunk in enumerate(chunks, start=1):
        chunk_metadata = doc.metadata.copy()
        chunk_metadata["id"] = chunk_metadata["_oid"] + "$" + str(id) + "$" + str(ic)
        chunk_metadata["document_id"] = str(id)
        chunk_metadata["document_summary"] = str(summ[0])
        chunks_with_mdata.append(
            Document(page_content=str(chunk), metadata=chunk_metadata)
        )

""" verify """
print(f"Number of total chunks with metadata: {len(chunks_with_mdata)}")

Connection successful!
ONNX model loaded.
Number of total chunks with metadata: 3


At this point, we have processed the documents and generated chunks with metadata. Next, we will create Oracle AI Vector Store with those chunks.

Here is the sample code how to do that:

In [55]:
# create Oracle AI Vector Store
vectorstore = OracleVS.from_documents(
    chunks_with_mdata,
    embedder,
    client=conn,
    table_name="oravs",
    distance_strategy=DistanceStrategy.DOT_PRODUCT,
)

""" verify """
print(f"Vector Store Table: {vectorstore.table_name}")

Vector Store Table: oravs


提供的示例演示了使用 DOT_PRODUCT 距离策略创建向量存储。用户可以使用 Oracle AI 向量存储的各种距离策略，详情请参阅我们的[全面指南](https://python.langchain.com/v0.1/docs/integrations/vectorstores/oracle/)。

现在嵌入向量已存储在向量存储中，建议建立索引以在执行查询期间提高语义搜索性能。

***注意*** 如果遇到"内存不足"错误，建议在数据库配置中增加 ***vector_memory_size***

以下是创建索引的示例代码：

In [56]:
oraclevs.create_index(
    conn, vectorstore, params={"idx_name": "hnsw_oravs", "idx_type": "HNSW"}
)

print("Index created.")

此示例演示了在'oravs'表中的嵌入向量上创建默认 HNSW 索引。用户可以根据具体需求调整各种参数。有关这些参数的详细信息，请参阅 [Oracle AI 向量搜索指南书](https://docs.oracle.com/en/database/oracle/oracle-database/23/vecse/manage-different-categories-vector-indexes.html)。

此外，可以创建各种类型的向量索引以满足不同需求。更多详细信息可以在我们的[全面指南](https://python.langchain.com/v0.1/docs/integrations/vectorstores/oracle/)中找到。


## 执行语义搜索
一切就绪！

我们已经成功处理了文档并将其存储在向量存储中，然后创建了索引以提高查询性能。我们现在已准备好进行语义搜索。

以下是此过程的示例代码：

In [58]:
query = "What is Oracle AI Vector Store?"
filter = {"document_id": ["1"]}

# Similarity search without a filter
print(vectorstore.similarity_search(query, 1))

# Similarity search with a filter
print(vectorstore.similarity_search(query, 1, filter=filter))

# Similarity search with relevance score
print(vectorstore.similarity_search_with_score(query, 1))

# Similarity search with relevance score with filter
print(vectorstore.similarity_search_with_score(query, 1, filter=filter))

# Max marginal relevance search
print(vectorstore.max_marginal_relevance_search(query, 1, fetch_k=20, lambda_mult=0.5))

# Max marginal relevance search with filter
print(
    vectorstore.max_marginal_relevance_search(
        query, 1, fetch_k=20, lambda_mult=0.5, filter=filter
    )
)

[Document(page_content='The database stores LOBs differently from other data types. Creating a LOB column implicitly creates a LOB segment and a LOB index. The tablespace containing the LOB segment and LOB index, which are always stored together, may be different from the tablespace containing the table. Sometimes the database can store small amounts of LOB data in the table itself rather than in a separate LOB segment.', metadata={'_oid': '662f2f257677f3c2311a8ff999fd34e5', '_rowid': 'AAAR/xAAEAAAAAnAAC', 'id': '662f2f257677f3c2311a8ff999fd34e5$3$1', 'document_id': '3', 'document_summary': 'Sometimes the database can store small amounts of LOB data in the table itself rather than in a separate LOB segment.\n\n'})]
[]
[(Document(page_content='The database stores LOBs differently from other data types. Creating a LOB column implicitly creates a LOB segment and a LOB index. The tablespace containing the LOB segment and LOB index, which are always stored together, may be different from th